# Project Proposal # 

## Introduction ##

Heart diseases refers to types of heart conditions in which their common symptoms vary from having heart attacks, arrhythmia (abnormal heart beats), to heart failure. Though these symptoms may exist for some people, many cannot be diagnosed unless experienced one of the conditionHeart diseases refers to types of heart conditions in which their common symptoms vary from having heart attacks, arrhythmia (abnormal heart beats), to heart failure. Though these symptoms may exist for some people, many cannot be diagnosed unless experienced one of the conditions above. Our question we would like to address is predicting the presence of heart disease by working with 4 variables as the predictors. The four variables are age, maximum heart rate, resting blood pressure, and cholesterol (These predictors are subjected to change as we move towards the end of the course as we learn how to find which are the best predictors). The accuracy of our model could be helpful as being the preliminary test to see if a patient needs further diagnosis before clinicians coming up with suitable treatment models for that patient.  For our project, we chose to focus on a dataset from UCI Machine Learning that contains data pulled from Hungary, Cleveland, Switzerland, and Longbeach that we found on Kaggle. https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset. 



The dataset includes 14 columns:
- age
- sex
- chest pain type (4 values; 0 = typical angina, 1 = atypical angina, 2 = non-anginal pain, 3 = asymptomatic)
- trestbps: resting blood pressure
- chol: serum cholestoral in mg/dl
- fbs: fasting blood sugar > 120 mg/dl
- restecg: resting electrocardiographic results (values 0,1,2)
- thalach: maximum heart rate achieved
- exang: exercise induced angina
- oldpeak: ST depression induced by exercise relative to rest
- slope: the slope of the peak exercise ST segment
- ca: number of major vessels (0-3) colored by flourosopy
- thal: 0 = normal; 1 = fixed defect; 2 = reversable defect
- target: presence of heart disease, 0 = False, 1 = True


In [5]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [6]:
heart_data <- read_csv(file = "heart.csv")

heart_data

Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
50,0,0,110,254,0,0,159,0,0.0,2,0,2,1
54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


In [7]:
# na_check <- heart_data |>
#     map_df(is.na) |>
#     map_df

In [8]:
heart_data  <- heart_data |>
    mutate(sex = as_factor(sex), target = as_factor(target), fbs = as_factor(fbs)) |>
    select(age, sex, chol, fbs, thalach, target)

colnames(heart_data) <- c("age", "sex", "cholesterol", "fasting_blood_sugar_greater_than_120", "max_heart_rate", "disease_present")

# changes 1 to male and 0 to female
levels(heart_data$sex) <- c("female", "male")
# changes disease_present to true/false (0 = false, 1 = true)
levels(heart_data$disease_present) <- c(FALSE, TRUE)
#changes fbs>120 to true/false (1 = true, 0, false)
levels(heart_data$fasting_blood_sugar_greater_than_120) <- c(FALSE, TRUE)

heart_data

age,sex,cholesterol,fasting_blood_sugar_greater_than_120,max_heart_rate,disease_present
<dbl>,<fct>,<dbl>,<fct>,<dbl>,<fct>
52,male,212,FALSE,168,FALSE
53,male,203,TRUE,155,FALSE
70,male,174,FALSE,125,FALSE
⋮,⋮,⋮,⋮,⋮,⋮
47,male,275,FALSE,118,FALSE
50,female,254,FALSE,159,TRUE
54,male,188,FALSE,113,FALSE


In [11]:
heart_split <- initial_split(heart_data, prop = 0.75, strata = disease_present)
heart_train <- training(heart_split)

heart_explore <- heart_train |>
    group_by(sex, disease_present) |>
    summarize(count = n(),
              avg_chol = mean(cholesterol),
              avg_age = round(mean(age)))

heart_explore

`summarise()` has grouped output by 'sex'. You can override using the `.groups`
argument.


sex,disease_present,count,avg_chol,avg_age
<fct>,<fct>,<int>,<dbl>,<dbl>
female,FALSE,63,277.0000,59
female,TRUE,171,255.5848,55
male,FALSE,311,247.3762,56
male,TRUE,223,229.7085,51


In [12]:
heart_explore_counts <- heart_train |>
    group_by(disease_present) |>
    summarize(count = n(),
              percent = (n() / nrow(heart_train)) * 100)

heart_explore_counts

disease_present,count,percent
<fct>,<int>,<dbl>
FALSE,374,48.69792
TRUE,394,51.30208
